# Softmax Regression

softmax模型可以用来给不同的对象分配概率。<br>
在训练更加复杂的模型时，最后一步也往往需要用softmax来分配概率。<br>

softmax回归分两步：
* 首先 对某个待分类对象属于某个类的“证据”相加求和
* 然后 将这个“证据”的和和转化为概率

例如计算一张图片是否属于某类。<br>
使用加权的方法来累积计算这张图是否属于某类的“证据”。<br>
如果图片的像素强有力的体现该图不属于某个类，则权重的数值为负数。<br>
相反，则权重的数值为正。<br>
还需要引入额外的“证据”，称之为偏置量(bias)<br>
因此对于给定的输入图片$x$是属于第$i$类的总体“证据”可以表示为：

$$
evidence_i = \sum_{i}W_{i,j}x_j + b_i
$$

其中$W_i$代表权重，$b_i$代表第$i$类的偏置量，$j$代表给定图片$x$的像素索引 用于求和<br>
然后利用softmax函数可以把这些“证据”转化成概率$y$

$$
y = softmax(evidence)
$$

给定一张图$x$，它对于每一个类别的吻合度可以被softmax函数转换成一个概率值<br>
softmax函数可以定义为：

$$
softmax(x) = normalize(exp(x))
$$

展开右边可以得到：

$$
softmax(x)_i = \frac{exp(x_i)}{\sum_{j}exp(x_j)}
$$

假设模型里的权值不可以是0或是负数，softmax会正则化这些权重值，<br>
是它们的总和等于1，以此构建一个有效率的概率分布。

$$
\left[
\begin{matrix}
y_1 \\
y_2 \\
y_3
\end{matrix}
\right] = softmax\left(
\begin{matrix}
W_{1,1}x_1 + W_{1,2}x_1 + W_{1,3}x_1 + b_1 \\
W_{2,1}x_2 + W_{2,2}x_2 + W_{2,3}x_2 + b_2 \\
W_{3,1}x_3 + W_{3,2}x_3 + W_{3,3}x_3 + b_3
\end{matrix}
\right)
$$

转换成用 矩阵乘法和向量相加来表示：

$$
\left[
\begin{matrix}
y_1 \\
y_2 \\
y_3
\end{matrix}
\right] = softmax\left(
\left[\begin{matrix}
W_{1,1}&W_{1,2}&W_{1,3} \\
W_{2,1}&W_{2,2}&W_{2,3} \\
W_{3,1}&W_{3,2}&W_{3,3}
\end{matrix}\right]
\cdot
\left[\begin{matrix}
x_1 \\
x_2 \\
x_3
\end{matrix}\right]
+
\left[\begin{matrix}
b_1 \\
b_2 \\
b_3
\end{matrix}\right]
\right)
$$

简化为：
$$
y = softmax(W_x+b)
$$

## 评估模型
为了训练模型，通常需要定义一个指标来评估模型的好坏。<br>
这个指标被称为成本(cost)或是损失(loss)，两者意思相同，然后尽量最小化这个指标。<br>
非常常见的成本函数是“交叉熵”(cross-entropy):

$$
H_{y'}(y) = -\sum_{i}y'_i\log(y_i)
$$

其中y是预测的概率分布。<br>
y‘是实际的分布。<br>

## 计算交叉熵
（重要）<br>
交叉熵不仅仅用来衡量单一的一对预测和真实值，<br>
而是所有图片的交叉熵的总和。<br>
对100个数据点的预测的表示比单一数据点的预测的表示能更好的描述模型的性能。

In [1]:
import tensorflow as tf
import input_data
import numpy as np
import time
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
np.set_printoptions(suppress=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# 1.参数设置

n_input => MNIST数据的图片特征个数为(784个)： 28 x 28 = 784<br>
n_classes => MNIST数据的种类个数为(10个)：0到9<br>
learning_rate => 学习速率：比较好的策略是先设置为0.25,然后在训练到20个Epoch时改为0.025。学习速率太大时会导致代价函数振荡；学习速率太小时会导致收敛过慢。<br>
batch_size => 批尺寸(批大小)：太大，权重的更新就不会那么频繁，优化过程太漫长；太小，计算的加速效果越不明显。<br>
total_batch => 1个epoch需要的迭代次数(550次)。5500 / 100 = 550<br>
training_epochs => 利用训练数据学习多少遍(10遍)。1个epoch意味着训练数据被用过1遍。<br>

In [2]:
n_input = 784
n_classes = 10
learning_rate = 0.25
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)
training_epochs = 10

# 2.创建模型

x => 是一个占位符，用来保存输入数据。类型时float32；第一个维度None表示可以是任何长度(输入图片的个数)；第二个维度784是每个图片展平后的向量。<br>
W => 是一个占位符，用来保存权重值。Variable常用来保存参数，计算中可以被修改，初始值为0。<br>
b => 是一个占位符，用来保存偏置量。<br>
y => 是预测出来的分类结果，还未softmax！！！<br>
y_pred => 是经过softmax之后的预测出来的分类结果。<br>
y_softm => 是一个占位符，用来保存输出数据的正确值(正确分类)。<br>
cross_entropy => 交叉熵，又可以叫做cost，有3种代码写作方式：第1种是直接按照公式写出的代码，如果出现log(0)的话结果就会变成Nan，不建议使用；第2种...with_logits用于较低版本的tf；第3种...with_logits_v2用于较新的tf。<br>
train_step => 学习步伐，又可叫做optimizer，永华方法使用梯度下降算法来最小化交叉熵，会自动使用反向传播法。<br>

In [3]:
x = tf.placeholder(tf.float32, [None, n_input])
W = tf.Variable(tf.zeros([n_input, n_classes]))
b = tf.Variable(tf.zeros([n_classes]))
y_pred = tf.matmul(x, W) + b
y_softmax = tf.nn.softmax(y_pred)
y_true = tf.placeholder(tf.float32, [None, n_classes])
# type 1:
# cross_entropy = tf.reduce_mean(
#     -tf.reduce_sum(y_true * tf.log(y_softmax), reduction_indices=[1]))
# type 2:
# cross_entropy = tf.reduce_mean(
#     tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))
# type 3:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true, logits=y_pred))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(
    cross_entropy)

# 3.为了评估模型所需要输出的数据

pred => 预测的分类结果。<br>
true => 正确的分类结果。<br>
correct_prediction => 对比预测结果和正确结果。<br>
accuracy => 正确率。<br>

In [4]:
pred = tf.argmax(y_pred, 1)
true = tf.argmax(y_true, 1)
correct_prediction = tf.equal(pred, true)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 4.训练模型并实时输出评估数据

循环训练10次，每次训练会随机抓取训练数据中100个数据点。<br>

In [5]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

start = time.time()
for epoch_i in range(training_epochs):
    ave_cost = 0
    for batch_i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        _, c = sess.run(
            [train_step, cross_entropy],
            feed_dict={
                x: batch_xs,
                y_true: batch_ys
            })
        ave_cost += c / total_batch
        # evidence and softmax data
        if (epoch_i == 0 and batch_i == 0) or (epoch_i == 9 and batch_i == 0):
            print("Epoch:%3d Batch:%5d " % (epoch_i, batch_i), )
            print("Details(0~5)--------------------------------------------")
            train_y_pred = y_pred.eval(feed_dict={
                x: batch_xs,
                y_true: batch_ys
            })
            train_y_softmax = y_softmax.eval(feed_dict={
                x: batch_xs,
                y_true: batch_ys
            })
            print("train_y_pred:", train_y_pred[:5])
            print("train_y_softmax:", train_y_softmax[:5])
            train_pred = pred.eval(feed_dict={x: batch_xs, y_true: batch_ys})
            train_true = true.eval(feed_dict={x: batch_xs, y_true: batch_ys})
            test_pred = pred.eval(feed_dict={
                x: mnist.test.images,
                y_true: mnist.test.labels
            })
            test_true = true.eval(feed_dict={
                x: mnist.test.images,
                y_true: mnist.test.labels
            })
            print("train_pred:", train_pred[:5])
            print("train_true:", train_true[:5])
            print("test_pred:", test_pred[:5])
            print("test_true:", test_true[:5])
            print("--------------------------------------------------------")
    if epoch_i % 1 == 0:
        train_acc = accuracy.eval(feed_dict={x: batch_xs, y_true: batch_ys})
        test_acc = accuracy.eval(feed_dict={
            x: mnist.test.images,
            y_true: mnist.test.labels
        })
        print("Epoch:%3d Batch:%5d " % (epoch_i,
                                        batch_i), "train_acc=%.5f" % train_acc,
              "test_acc=%.5f" % test_acc, "train_cost=%.9f" % ave_cost)
end = time.time()
print("Process Time(s):{:.2f}".format(end - start))

# result
print(
    sess.run(
        accuracy, feed_dict={
            x: mnist.test.images,
            y_true: mnist.test.labels
        }))
sess.close()

Epoch:  0 Batch:    0 
Details(0~5)--------------------------------------------
train_y_pred: [[-0.11002736  0.18487297  0.44305825 -0.38023573 -0.18479885 -0.2683453
   0.25828344  0.05070856 -0.0324292   0.03891305]
 [-0.30482468  0.19426018  1.136788   -0.6069247  -0.33325022 -0.52722865
   0.44119087  0.163528   -0.25636628  0.09282692]
 [-0.13573974  0.32633132  0.27970797 -0.21609351 -0.121985   -0.16661927
   0.06935494  0.02337473 -0.07345591  0.0151243 ]
 [ 0.04100353 -0.1082223   0.26374707 -0.49610126 -0.24050018 -0.08761284
   0.45472875  0.12797147 -0.15274322  0.1977284 ]
 [-0.25817424  0.09295582  0.636666   -0.6275138  -0.23226711 -0.3915826
   0.9120573   0.09065467 -0.26626045  0.04346411]]
train_y_softmax: [[0.08705255 0.11691092 0.15135038 0.06644028 0.08078089 0.07430617
  0.12581627 0.10223231 0.09407667 0.10103352]
 [0.0639776  0.10538474 0.27046582 0.04729635 0.06218461 0.05121994
  0.134902   0.1021953  0.0671542  0.09521949]
 [0.08594126 0.13641955 0.1302052  